In [20]:
import yfinance as yf
import pandas as pd
import numpy as np

# Download historical stock data
assets = ["LQD", "DBC", "SPY", "VNQ", "SHY"]
start_date = "1990-01-01"
end_date = "2023-08-11"
data = yf.download(assets, start=start_date, end=end_date)["Adj Close"]

# Calculate returns
returns = data.pct_change().dropna()

cash_prices = data["SHY"]
asset_prices = data.drop(columns=["SHY"])

# Define a function for calculating channel positions
def pct_channel_position(prices, day_lookback=60, lower_pct=0.25, upper_pct=0.75):
    rolling_max = prices.rolling(window=day_lookback).max()
    rolling_min = prices.rolling(window=day_lookback).min()
    
    upper_channels = rolling_min + (rolling_max - rolling_min) * upper_pct
    lower_channels = rolling_min + (rolling_max - rolling_min) * lower_pct

    positions = pd.DataFrame(index=prices.index, columns=prices.columns, dtype=float)
    positions[prices > upper_channels] = 1
    positions[prices < lower_channels] = -1
    positions = positions.ffill().fillna(0)
    
    return positions

# Calculate channel positions
d60 = pct_channel_position(asset_prices)
d120 = pct_channel_position(asset_prices, day_lookback=120)
d180 = pct_channel_position(asset_prices, day_lookback=180)
d252 = pct_channel_position(asset_prices, day_lookback=252)

composite_position = (d60 + d120 + d180 + d252) / 4

# Select positions at the end of each month
composite_months = composite_position.resample('M').last()

# Calculate returns
returns = asset_prices.pct_change().dropna()

# Calculate rolling standard deviations
rolling_sd20 = returns.rolling(window=20).std().resample('M').last()

# Calculate weights
weight = composite_months * (1 / (rolling_sd20*100))
weight_per_date = weight.abs().sum(axis=1)
rp_weight = np.abs(weight).div(weight_per_date, axis='index')
rp_weight[composite_months < 0] = 0

weight_date=weight.sum(axis=1)
weight = weight.div(weight_per_date, axis=0)
weight["CASH"] = 1 - weight.sum(axis=1, skipna=True)

# Calculate equal-weighted weights
ew_weight = np.abs(composite_months) / np.abs(composite_months).sum(axis=1, skipna=True, min_count=1)
ew_weight[composite_months < 0] = 0
ew_weight["CASH"] = 1 - ew_weight.sum(axis=1, skipna=True)

# Calculate portfolio returns
rp_rets = (returns * weight).sum(axis=1, skipna=True)
ew_rets = (returns * ew_weight).sum(axis=1, skipna=True)

[*********************100%***********************]  5 of 5 completed


In [21]:
weight

,DBC,LQD,SPY,VNQ,CASH
Date,,,,,
1993-01-31,NaN,NaN,NaN,NaN,1.000000
1993-02-28,NaN,NaN,NaN,NaN,1.000000
1993-03-31,NaN,NaN,NaN,NaN,1.000000
1993-04-30,NaN,NaN,NaN,NaN,1.000000
1993-05-31,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...
2023-04-30,-0.212259,0.400279,0.225131,-0.162330,0.749179
2023-05-31,-0.159314,0.428361,0.236517,-0.175808,0.670243
2023-06-30,-0.233753,0.265225,0.388552,-0.112469,0.692445


In [14]:
weight = composite_months * (1 / (rolling_sd20*100))
weight_per_date = weight.abs().sum(axis=1)
rp_weight = np.abs(weight).div(weight_per_date, axis='index')
rp_weight[composite_months < 0] = 0
rp_weight[-20:]
weight

,DBC,LQD,SPY,VNQ
Date,,,,
1993-01-31,NaN,NaN,NaN,NaN
1993-02-28,NaN,NaN,NaN,NaN
1993-03-31,NaN,NaN,NaN,NaN
1993-04-30,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN
...,...,...,...,...
2023-04-30,-1.184557,2.233839,1.256392,-0.905917
2023-05-31,-0.840057,2.258737,1.247151,-0.927032
2023-06-30,-0.908980,1.031354,1.510936,-0.437350


In [168]:
np.abs(weight).div((weight.abs().sum(axis=1)), axis='index')


,DBC,LQD,SPY,VNQ
Date,,,,
1993-01-31,NaN,NaN,NaN,NaN
1993-02-28,NaN,NaN,NaN,NaN
1993-03-31,NaN,NaN,NaN,NaN
1993-04-30,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN
...,...,...,...,...
2023-04-30,0.212259,0.400279,0.225131,0.162330
2023-05-31,0.159314,0.428361,0.236517,0.175808
2023-06-30,0.233754,0.265224,0.388553,0.112469
